In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

import reader
import os
import pydot_ng
import graphviz

In [2]:
file_name = 'select_description_from_sys_district.csv'
if reader.check_file(file_name, url=''):
    hotelDescs = reader.read_data_as_array(file_name)
else:
    quit()

文件  select_description_from_sys_district.csv 存在。盛宴大幕拉开了。
段落处理完毕


In [3]:
print(len(hotelDescs))
print(hotelDescs[1])

3520
2018年开业2018年装修143间房。云和夜泊酒店(上海浦东国际机场店)，位于浦东新区祝桥秋亭路，毗邻上海浦东国际机场，车程20分钟，距上海迪士尼18分钟车程，距上海野生动物园21分钟车程。酒店地理位置优越，并配备大巴、中巴、豪华轿车接送客户。酒店拥有房间98间，大床30间，行政大床15间，标间30间，亲子房23间。


In [4]:
hotelText = reader.read_data_as_text(file_name)
vocab = sorted(set(hotelText))
vocab_size = len(vocab)
print('有{}个不重复字符'.format(len(vocab)))

有3014个不重复字符


In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
hotelText_as_int = np.array([char2idx[c] for c in hotelText])

#for char, _ in zip(char2idx, range(10)):
#    print('{:4s}: {:3d}'.format(repr(char), char2idx[char]))

In [13]:
def build_dataset(documents):
    chars = []
    # 这是一些列表．外部列表表示每个文档,内部列表表示给定文档中的单词     
    data_list = []
  
    for d in documents:
        chars.extend(d)
    print('找到 %d 个单词.'%len(chars))
    count = []
    # 根据频率对bigram进行排序（最高的排在第一位）
    count.extend(collections.Counter(chars).most_common())
    
    # 通过给出字典的当前长度为每个bigram创建一个ID
    # 并将该元素添加到字典中
    # 从分配给文本中稀有单词的“UNK”开始    
    dictionary = dict({'UNK':0})
    for char, c in count:
        # 如果统计的频率超过10，则仅增加一个bigram到字典中
        if c > 4:
            dictionary[char] = len(dictionary)    
    
    unk_count = 0
    # 遍历所有文本,我们将每个字符串形式的单词替换为单词的ID     
    for d in documents:
        data = list()
        for char in d:
            # 如果一个单词存在于词典中,则使用该单词ID,否则使用特殊标记“UNK”的ID            
            if char in dictionary:
                index = dictionary[char]        
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)
            
        data_list.append(data)
        
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data_list, count, dictionary, reverse_dictionary

global data_list, count, dictionary, reverse_dictionary,vocabulary_size

# 打印数据一些的统计信息
data_list, count, dictionary, reverse_dictionary = build_dataset(hotelText)
print('常用的词 (+UNK)', count[:5])
print('不常用的词 (+UNK)', count[-15:])
print('样本数据', data_list[0][:10])
print('样本数据', data_list[1][:10])
print('词汇量: ',len(dictionary))
vocabulary_size = len(dictionary)

找到 894831 个单词.
常用的词 (+UNK) [('，', 33827), ('、', 22474), ('。', 17855), ('的', 16469), ('店', 14173)]
不常用的词 (+UNK) [('棒', 1), ('瓢', 1), ('俚', 1), ('狼', 1), ('猬', 1), ('涨', 1), ('℃', 1), ('催', 1), ('厘', 1), ('逢', 1), ('庠', 1), ('剥', 1), ('｡', 1), ('辨', 1), ('蔼', 1)]
样本数据 [11]
样本数据 [6]
词汇量:  2155


In [ ]:
for hotelDesc in hotelDescs:
    seq_length = len(hotelDesc)
    example_per_epoch = len(hotelText) // seq_length

In [5]:

model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=vocab_size, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units and softmax activation.
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          192896    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 293,002
Trainable params: 293,002
Non-trainable params: 0
_________________________________________________________________


In [6]:
#tf.keras.utils.plot_model(model, 'a.png')  出不来

InvocationException: GraphViz's executables not found